In [282]:
import csv
from datetime import date, timedelta
from pickle import dump, load
import calendar
import numpy as np
import pandas as pd
from yahoo_finance import Share



STOCK_INDEX_FILES = ['data/DJIA.csv', 'data/NASDAQCOM.csv', 'data/SP500.csv']
INDEX_NAMES = {'Nasdaq': '^IXIC', 'Dow Jones': '^DJI', 'S&P 500': '^GSPC'}


# Labels
NO_CURRENT_VALUE = 'No current value'
NO_NEXT_VALUE = 'No next value'
UP = 'Up'
DOWN = 'Down'

def generate_date_dicts():
    for name in STOCK_INDEX_FILES:
        with open(name, 'r') as f:
            date_dict = {}
            reader = csv.reader(f)
            reader.__next__() # get rid of header
            for row in reader:
                if row[1] == '.': # no entry in stock file
                    continue
                d = date(*list(map(int, row[0].split('-')))) # format date into datetime object
                value = float(row[1])
                date_dict[d] = value

        with open(name.replace('.csv', NAME_EXTENSION), 'wb') as f:
            dump(date_dict, f, protocol=2)


def get_date_dict(path):
    date_dict = {}
    with open(path, 'r') as f:
            reader = csv.reader(f)
            reader.__next__() # get rid of header
            for row in reader:
                if row[1] == '.': # no entry in stock file
                    continue
                d = date(*list(map(int, row[0].split('-')))) # format date into datetime object
                value = float(row[1])
                date_dict[d] = value
    return date_dict

def get_date_window(date_dict, center_date, window_size=4):
    values = []
    i = 0
    while len(values) < window_size:
        time_delta = timedelta(days=i, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
        d = center_date - time_delta
        if d in date_dict:
            values.append(date_dict[d])
        i += 1
    values.reverse()
    return values

def get_next_date_value(date_dict, center_date):
    time_delta = timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
    d = center_date + time_delta
    if d not in date_dict:
        return None
    return date_dict[d]

def get_wh_data():
    month_to_int = {name: num for num, name in enumerate(calendar.month_name) if num}
    dates = []
    titles = []
    bodies = []
    with open('data/WH_posts.csv', 'r') as f:
        reader = csv.reader(f)
        reader.__next__()
        for row in reader:
            d = row[0].replace(',','').split()
            d = date(*[int(d[2]), month_to_int[d[0]], int(d[1])])
            dates.append(d)
            titles.append(row[1])
            bodies.append(row[2])

    data = {'dates':dates, 'titles':titles, 'bodies':bodies}
    with open('data/WH_posts_structured', 'wb') as f:
        dump(data, f, protocol=2)
        
    return dates, titles, bodies


def generate_labels(date_dict, wh_dates):
    labels = []
    for d in wh_dates:
        if d in date_dict:
            cur_value = date_dict[d]
        else:
            labels.append(NO_CURRENT_VALUE)
            continue

        next_value = get_next_date_value(date_dict, d)
        if next_value:
            labels.append(UP if (next_value - cur_value) >= 0 else DOWN)
        else:
            labels.append(NO_NEXT_VALUE)
    return labels

def filter_labels(labels):
    labels = np.array(labels)
    labels = labels[np.where(labels != NO_CURRENT_VALUE)[0]]
    labels = labels[np.where(labels != NO_NEXT_VALUE)[0]]
    return labels

def get_stock_values(stock_abbrv):
    """
    Given the stock abbrevation, this function
    will pull from Yahoo Finance the history of that 
    stock from 2016-12-25 to present day
    """
    share = Share(stock_abbrv)
    share_history = share.get_historical('2016-12-25', date.isoformat(date.today()))
    df = pd.DataFrame([[s['Date'], float(s['Close'])] for s in share_history], columns=['Date', 'Value'])
    df['Date'] = pd.to_datetime(df['Date'])
    return df
    
def merge_texts(wh_df):
    """
    Merges the text fields of posts
    that are on the same date.
    """
    date_to_bodies = {}
    date_to_titles = {}
    dates = pd.to_datetime(wh_df['Date'])
    titles = wh_df['Title']
    bodies = wh_df['Body']
    for i in range(len(dates)):
        if dates[i] not in date_to_titles:
            date_to_titles[dates[i]] = ''
            date_to_bodies[dates[i]] = ''
        date_to_titles[dates[i]] += titles[i] + ' '
        date_to_bodies[dates[i]] += bodies[i] + ' '
        
    title_df = pd.DataFrame(list(date_to_titles.items()), columns=['Date', 'Title'])
    body_df = pd.DataFrame(list(date_to_bodies.items()), columns=['Date', 'Body'])
    wh_df = pd.merge(title_df, body_df, how='inner', on=['Date'])
    
    return wh_df

def create_dataset():
    """
    Pulls closing values from Yahoo finance and
    matches those values to white house posts using
    the date field. Returns a dataframe for easy
    manipulation.
    """
    wh_df = pd.read_csv('data/WH_posts.csv')
    wh_df['Date'] = pd.to_datetime(wh_df['Date'])
        
    stock_dfs = {name:get_stock_values(INDEX_NAMES[name]) for name in INDEX_NAMES}
    processed_stock_dfs = []
    for name in stock_dfs:
        stock_df = stock_dfs[name]
        dates = stock_df['Date']
        values = stock_df['Value'].rename(name + ' Value')
        stock_df = pd.concat([dates, values], axis=1, join_axes=[dates.index])
        stock_df.sort_values(by='Date')
        
        stock_df[name + ' Delta'] = stock_df[name + ' Value'].diff(periods=1)
        processed_stock_dfs.append(stock_df)     

        
    wh_df['Date'] = pd.to_datetime(wh_df['Date'])
    stock_df = processed_stock_dfs[0]
    for i in range(1, len(stock_dfs)):
        stock_df = pd.merge(stock_df, processed_stock_dfs[i], how='inner', on=['Date'])
    
    dataset = pd.merge(wh_df, stock_df, how='inner', on=['Date'])
    
    return dataset.sort_values(by='Date').reset_index(drop=True)
    
    
    

How do we want to handle labeling? Do we want to average the three indexes together or generate 3 different labels?

In [22]:
wh_dates, wh_titles, wh_bodies = get_wh_data()
dj_dict = get_date_dict('data/DJIA.csv')
nd_dict = get_date_dict('data/NASDAQCOM.csv')
sp_dict = get_date_dict('data/SP500.csv')

dj_labels = filter_labels(generate_labels(dj_dict, wh_dates))
nd_labels = filter_labels(generate_labels(nd_dict, wh_dates))
sp_labels = filter_labels(generate_labels(sp_dict, wh_dates))

In [24]:
print("There are %d labels total" %len(dj_labels))
print("No label: %d" % (len(np.where(dj_labels == NO_CURRENT_VALUE)[0]) + len(np.where(dj_labels == NO_NEXT_VALUE)[0])))
print("There are %d positive labels" % (499-159))

index_differences = np.where(dj_labels != nd_labels)[0]
print(len(np.where(dj_labels != nd_labels)[0]))
print(len(np.where(dj_labels != sp_labels)[0]))
print(len(np.where(nd_labels != sp_labels)[0]))
print(dj_labels[index_differences])


There are 340 labels total
No label: 0
There are 340 positive labels
121
64
77
['Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down'
 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down'
 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down'
 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down'
 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down'
 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down'
 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up'
 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Down' 'Down' 'Down'
 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down'
 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Down'
 'Down' 'Down' 'Down' 'Down' 'Down' 'Down' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up'
 'Up' 'Up' 'Up' 'Up']


- Figure out how to pull stock data automatically

In [283]:
dataset = create_dataset()
dataset

,Date,Title,Body,S&P 500 Value,S&P 500 Delta,Nasdaq Value,Nasdaq Delta,Dow Jones Value,Dow Jones Delta
0,2017-01-23,Presidential Memorandum Regarding the Hiring F...,MEMORANDUM FOR THE HEADS OF EXECUTIVE DEPARTME...,2265.199951,-14.870117,5552.939941,-48.020020,19799.849609,-112.861329
1,2017-01-23,Presidential Memorandum Regarding Withdrawal o...,MEMORANDUM FOR THE UNITED STATES TRADE REPRESE...,2265.199951,-14.870117,5552.939941,-48.020020,19799.849609,-112.861329
2,2017-01-24,Memorandum: Implementation of Regulatory Freeze,M-17-16MEMORANDUM FOR THE HEADS AND ACTING HEA...,2280.070068,-18.300049,5600.959961,-55.379883,19912.710938,-155.798828
3,2017-01-24,Presidential Memorandum Regarding Construction...,MEMORANDUM FOR THE SECRETARY OF THE ARMYSUBJEC...,2280.070068,-18.300049,5600.959961,-55.379883,19912.710938,-155.798828
4,2017-01-24,Presidential Memorandum Regarding Construction...,MEMORANDUM FOR THE SECRETARY OF STATE ...,2280.070068,-18.300049,5600.959961,-55.379883,19912.710938,-155.798828
5,2017-01-24,Executive Order Expediting Environmental Revie...,EXECUTIVE ORDER- - - - - - -EXPEDITING ENVIRON...,2280.070068,-18.300049,5600.959961,-55.379883,19912.710938,-155.798828
6,2017-01-24,Presidential Memorandum Regarding Construction...,MEMORANDUM FOR THE SECRETARY OF COMMERCESUBJEC...,2280.070068,-18.300049,5600.959961,-55.379883,19912.710938,-155.798828
7,2017-01-24,Presidential Memorandum Streamlining Permittin...,"January 24, 2017MEMORANDUM FOR THE HEADS OF EX...",2280.070068,-18.300049,5600.959961,-55.379883,19912.710938,-155.798828
8,2017-01-24,President Trump Takes Action to Expedite Prior...,"WASHINGTON, DC President Donald J. Trump toda...",2280.070068,-18.300049,5600.959961,-55.379883,19912.710938,-155.798828
9,2017-01-24,Statement from the White House Press Office,"""It is a longstanding tradition that the child...",2280.070068,-18.300049,5600.959961,-55.379883,19912.710938,-155.798828
